In [1]:
from typing import Optional
from pydantic import BaseModel
from pyfuseki import FusekiUpdate
import httpx

In [2]:
class Label(BaseModel):
    value: str
    lang: Optional[str]

class Element(BaseModel):
    type: str
    elementValue: Label 

class Variant(BaseModel): 
    type: str
    elementList: list[Element]


class EditVariant(BaseModel): 
    old: Variant
    new: Variant


In [3]:
obj = {
    "old": { 
  "type": "PersonalName",
  "elementList": [
    {
      "type": "FullNameElement",
      "elementValue": {
        "value": "Sousa Saramago, José de"
      } }
  ]
},
  "new": { 
  "type": "PersonalName",
  "elementList": [
    {
      "type": "FullNameElement",
      "elementValue": {
        "value": "MODIFICADO"
      } }
  ]
}
}

request = EditVariant(**obj)
request

EditVariant(old=Variant(type='PersonalName', elementList=[Element(type='FullNameElement', elementValue=Label(value='Sousa Saramago, José de', lang=None))]), new=Variant(type='PersonalName', elementList=[Element(type='FullNameElement', elementValue=Label(value='MODIFICADO', lang=None))]))

In [7]:
label = [i.elementValue.value for i in request.old.elementList]
label

['Sousa Saramago, José de']

In [4]:
from api.src.function.authorities.makeVariant import MakeSparqlVariant


qVariant = MakeSparqlVariant("https://bibliokeia.com/authorities/PersonalName/bkau-1", request.old)
print(qVariant)

<https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant ?variant .
        ?variant rdf:type  madsrdf:Variant , madsrdf:PersonalName .
        ?hasVariant madsrdf:variantLabel ?variantLabel .
        ?variant madsrdf:elementList ?elementList . 
        ?elementList rdf:first ?e .
        ?e rdf:type madsrdf:FullNameElement . 
        ?e madsrdf:elementValue "Sousa Saramago, José de" . 
        ?elementList rdf:rest rdf:nil .
        


In [6]:
authority = "https://bibliokeia.com/authorities/PersonalName/bkau-1"

def EditVariantJena(authority, request):
    
        vOld = MakeSparqlVariant(authority, request.old)
        vNew = MakeSparqlVariant(authority, request.new)
        variant = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>    
                        WITH <{authority}> 
                        DELETE {{ {vOld} }}
                        INSERT {{ {vNew}  }}
                        WHERE {{{vOld}  }}"""

        upAuthorities = FusekiUpdate('http://localhost:3030', 'authorities')

        r = upAuthorities.run_sparql(variant)
        return r.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [9]:
from api.src.function.authorities.makeElement import MakeElement, MakeLabel

In [7]:
elementList = MakeElement(request.elementList)
print(elementList)

[ a madsrdf:FullNameElement ;
                madsrdf:elementValue "García Márquez, Gabriel," ] [ a madsrdf:DateNameElement ;
                madsrdf:elementValue "1928-" ]


In [12]:
MakeLabel(request.elementList)

'García Márquez, Gabriel, 1928-'

In [15]:
def MakeVariant(request):

    elements = [
        f"""<https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant ?variant .
        ?variant rdf:type  madsrdf:Variant , madsrdf:{request.variantType} .
        ?hasVariant madsrdf:variantLabel ?variantLabel .
        ?variant madsrdf:elementList ?elementList . \n"""
    ]
    for i in request.elements:
        index = request.elements.index(i)
        if index == 0:
            rest = f'?rest{index+1}'
            e = f"""?elementList rdf:first ?e{index} .
        ?e{index} rdf:type madsrdf:{i.elementType} . 
        ?e{index} madsrdf:elementValue "{i.value}" . 
        ?elementList rdf:rest {rest} . \n"""
            elements.append(e)
        else:
            e = f"""{rest} rdf:first ?e{index} .
        ?e{index} rdf:type madsrdf:{i.elementType} . 
        ?e{index} madsrdf:elementValue "{i.value}" .\n"""
            if i == request.elements[-1]:
                nil = f"{rest} rdf:rest rdf:nil"
                elements.append(e)
                elements.append(nil)
                break
            elements.append(e)
            rest = f'?rest{index+1}'
        

    variant = " ".join(elements)   
    return variant  

v = MakeVariant(request)
print(v)
    


AttributeError: 'Variant' object has no attribute 'variantType'

In [66]:
v = "PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>\n        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\n        WITH <https://bibliokeia.com/authorities/PersonalName/bkau-1> \n        DELETE {<https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant ?variant .\n                ?variant rdf:type  madsrdf:Variant , madsrdf:PersonalName .\n                ?hasVariant madsrdf:variantLabel ?variantLabel .\n                ?variant madsrdf:elementList ?elementList . \n ?elementList rdf:first ?e0 .\n            ?e0 rdf:type madsrdf:FullNameElement . \n            ?e0 madsrdf:elementValue \"Gacxia Mackết, Gabriel,\" . \n            ?elementList rdf:rest ?rest1 . \n ?rest1 rdf:first ?e1 .\n            ?e1 rdf:type madsrdf:DateNameElement . \n            ?e1 madsrdf:elementValue \"1927-2014\" .\n ?rest1 rdf:rest rdf:nil}\n        WHERE { <https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant ?variant .\n                ?variant rdf:type  madsrdf:Variant , madsrdf:PersonalName .\n                ?hasVariant madsrdf:variantLabel ?variantLabel .\n                ?variant madsrdf:elementList ?elementList . \n ?elementList rdf:first ?e0 .\n            ?e0 rdf:type madsrdf:FullNameElement . \n            ?e0 madsrdf:elementValue \"Gacxia Mackết, Gabriel,\" . \n            ?elementList rdf:rest ?rest1 . \n ?rest1 rdf:first ?e1 .\n            ?e1 rdf:type madsrdf:DateNameElement . \n            ?e1 madsrdf:elementValue \"1927-2014\" .\n ?rest1 rdf:rest rdf:nil}"
print(v)

PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        WITH <https://bibliokeia.com/authorities/PersonalName/bkau-1> 
        DELETE {<https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant ?variant .
                ?variant rdf:type  madsrdf:Variant , madsrdf:PersonalName .
                ?hasVariant madsrdf:variantLabel ?variantLabel .
                ?variant madsrdf:elementList ?elementList . 
 ?elementList rdf:first ?e0 .
            ?e0 rdf:type madsrdf:FullNameElement . 
            ?e0 madsrdf:elementValue "Gacxia Mackết, Gabriel," . 
            ?elementList rdf:rest ?rest1 . 
 ?rest1 rdf:first ?e1 .
            ?e1 rdf:type madsrdf:DateNameElement . 
            ?e1 madsrdf:elementValue "1927-2014" .
 ?rest1 rdf:rest rdf:nil}
        WHERE { <https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant ?variant .
                ?variant rdf:type  madsrdf:Variant , madsrdf

In [69]:
d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = upAuthorities.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [70]:
authority = 'http://id.loc.gov/authorities/names/n79063441'
url = f'http://localhost:8000/authorities/import/loc/agents?authority={authority}'
r = httpx.get(url)
r.status_code

200

In [73]:
r.json()

{'type': 'PersonalName',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '2023-06-01',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-06-01T10:05:47',
  'identifiedBy': [{'type': 'Lccn',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'n79063441'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'elementList': [{'type': 'FullNameElement',
   'elementValue': {'value': 'García Márquez, Gabriel,', 'lang': None}},
  {'type': 'DateNameElement',
   'elementValue': {'value': '1927-2014', 'lang': None}}],
 'fullerName': {'type': 'PersonalName',
  'elementValue': {'value': 'Gabriel José de la Concordia', 'lang': None}},
 'birthDate': '1927-03-06',
 'birthPlace': 'Aracataca (Colombia)',
 'deathDate': '2014-04-17',
 'hasAffiliation': None

In [74]:
r.json()['hasVariant']

[{'type': 'PersonalName',
  'elementList': [{'type': 'FullNameElement',
    'elementValue': {'value': 'García Márquez, Gabriel,', 'lang': None}},
   {'type': 'DateNameElement',
    'elementValue': {'value': '1928-', 'lang': None}}]},
 {'type': 'PersonalName',
  'elementList': [{'type': 'FullNameElement',
    'elementValue': {'value': 'Jiaxiya Maerkesi, Jiafulieer,', 'lang': None}},
   {'type': 'DateNameElement',
    'elementValue': {'value': '1927-2014', 'lang': None}}]},
 {'type': 'PersonalName',
  'elementList': [{'type': 'FullNameElement',
    'elementValue': {'value': 'Gacxia Mackết, Gabriel,', 'lang': None}},
   {'type': 'DateNameElement',
    'elementValue': {'value': '1927-2014', 'lang': None}}]},
 {'type': 'PersonalName',
  'elementList': [{'type': 'FullNameElement',
    'elementValue': {'value': 'García Márquez, G.', 'lang': None}},
   {'type': 'FullNameElement',
    'elementValue': {'value': '(Gabriel),', 'lang': None}},
   {'type': 'DateNameElement',
    'elementValue': {'va

In [72]:
res = httpx.post("http://localhost:8000/authorities/agents", data=r.json())
res

<Response [307 Temporary Redirect]>

{'statusCode': 200, 'message': 'Update succeeded'}

In [31]:
variant = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        WITH <https://bibliokeia.com/authorities/PersonalName/bkau-1>

        DELETE {{
<https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant ?hasVariant .
    ?hasVariant rdf:type madsrdf:Variant, madsrdf:{request.variantType} .
    ?hasVariant madsrdf:variantLabel ?variantLabel .
    ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?first .
    ?first madsrdf:elementValue "{request.name.value}" .
    ?first rdf:type madsrdf:{request.name.elementType} .   
    ?elementList rdf:rest rdf:nil .  
     {  '?elementList rdf:rest ?rest .' if request.date else '' } 
     {  '?rest rdf:rest rdf:nil . ' if request.date else '' } 
     {  '?rest rdf:first ?date . ' if request.date else '' } 
     {  '?date rdf:type madsrdf:DateNameElement . ' if request.date else '' } 
     {  f'?date madsrdf:elementValue "{request.date.value}"' if request.date else '' }     
        }}

    WHERE {{ 
       <https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant ?hasVariant .
    ?hasVariant rdf:type madsrdf:Variant, madsrdf:{request.variantType} .
    ?hasVariant madsrdf:variantLabel ?variantLabel .
    ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?first .
    ?first madsrdf:elementValue "{request.name.value}" .
    ?first rdf:type madsrdf:{request.name.elementType} . 
    ?elementList rdf:rest rdf:nil .  
     {  '?elementList rdf:rest ?rest .' if request.date else '' } 
     {  '?rest rdf:rest rdf:nil . ' if request.date else '' } 
     {  '?rest rdf:first ?date . ' if request.date else '' } 
     {  '?date rdf:type madsrdf:DateNameElement . ' if request.date else '' } 
     {  f'?date madsrdf:elementValue "{request.date.value}"' if request.date else '' }  
        }}"""

r = upAuthorities.run_sparql(variant)
r.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [8]:
delete = """PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
DELETE DATA
{ GRAPH <https://bibliokeia.com/authorities/PersonalName/bkau-1> { 
    <https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant [
    rdf:type madsrdf:Variant , madsrdf:PersonalName ;
    madsrdf:elementList (
        [ rdf:type  madsrdf:FullNameElement ;
          madsrdf:elementValue  "FLAVIO GUSTAVO" ]
        [ rdf:type madsrdf:DateNameElement ;
          madsrdf:elementValue  "1821-1880" ] ) ;
  madsrdf:variantLabel  "Flaubert, Gustaw,, 1821-1880" ]
      } }"""

In [9]:
upAcervo = FusekiUpdate('http://localhost:3030', 'authorities')

r = upAcervo.run_sparql(delete)
r.convert()

QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Line 4, column 81: Blank nodes not allowed in DELETE templates\n'

In [ ]:
madsrdf:hasVariant          [ rdf:type              madsrdf:Variant , madsrdf:PersonalName ;
                                      madsrdf:elementList   ( [ rdf:type              madsrdf:FullNameElement ;
                                                                madsrdf:elementValue  "FLAVIO GUSTAVO"
                                                              ]
                                                              [ rdf:type              madsrdf:DateNameElement ;
                                                                madsrdf:elementValue  "1821-1880"
                                                              ]
                                                            ) ;
                                      madsrdf:variantLabel  "Flaubert, Gustaw,, 1821-1880"
                                    ] ;

<https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant ?variant .
    ?variant rdf:type  madsrdf:Variant , madsrdf:PersonalName .
    ?variant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?e1 .
    ?e1 rdf:type madsrdf:FullNameElement . 
    ?e1 madsrdf:elementValue "Jiaxiya Maerkesi, Jiafulieer," .
    ?elementList rdf:rest ?rest .
    ?rest rdf:first ?e2 .
    ?e2 madsrdf:elementValue "1927-2014" .
    ?e2 rdf:type madsrdf:DateNameElement .
    ?rest rdf:rest rdf:nil